In [1]:
import sys, os
# sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import scipy.integrate as si
import emcee
import corner
import getdist
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from HzGPR import H_GPR
from HzFid import H_th
from scipy.optimize import minimize
from getdist import plots, MCSamples


In [5]:
    param_cosmo_fid = [0.2657, 0.00493, 0, 0.704, -1]

# Função $\chi^{2}$ da Fração de Massa e Cálculo do Erro Médio do modelo 1:

Starting from the logarithm of the likelihood, we have:

$$
    \ln\,\mathcal{P} \left( y^{\text{th} }\,|\,y^{ \text{ob} },\sigma_{y^{ \text{ob} }}\right) \propto -\frac{1}{2}\chi^2 \left( y^{ \text{th} }\,|\,y^{ \text{ob} },\sigma_{y^{\text{ob} }}\right)  =
    -\frac{1}{2} \sum_n \left[
        \frac{(y^{\text{th} }(z_n)-y^{\text{ob} }_{n})^2}{\sigma_{y^{\text{ob} }_n}^2}
    \right] \,\,\, ,
$$

So, for the first two models, we will have:

$$
y^{\text{th} }(z_n) = \gamma_0(1+\eta_0z)^{2} \,\,\,\, .
$$

Thus, for this model, the $\chi^{2}$ will be:

$$
\chi^{2}=\sum_{i=1}^{40}\frac{ \left[ \gamma_{0} (1+\eta _{0}z)^{2}-
\frac{f_{\text{gas},\ i}^{\text{obs} } }{ \bar{f}_{gas} }\,\,\, \right] ^{2}}{\sigma _{\text{
tot},\ i}^{2}}\text{ .}
$$

The total error, $\sigma^{2}_{\text{tot}}$, is written as:

$$
\sigma^{2}_{\text{tot}} =  \sum_{n} \, \left[ \frac{\partial \, G(X_{n})}{\partial X_{n}}\right]\sigma^{2}_{X_{n}}
$$

Considering that $G(X_{n})= - \frac{ f^{\text{obs}}_{\text{gas}} }{ \bar{f} }$, we have:

$$ 
\sigma^{2}_{\text{tot} } = \left[ \frac{\partial}{\partial f^{ \text{obs}}_{\text{gas}} } \left( - \frac{ f^{\text{obs}}_{\text{gas}} }{ \bar{f}}\right) \right]^{2} \sigma^{2}_{f^{\text{obs}}_{\text{gas}}} + \left[ \frac{\partial}{\partial \bar{f} } \left( - \frac{ f^{\text{obs}}_{\text{gas}} }{ \bar{f}}\right) \right]^{2} \sigma^{2}_{\bar{f}}
$$

And thus,

$$ 
 \sigma^{2}_{\text{tot} } = \frac{1}{\bar{f}^{2} } \sigma^{2}_{f^{\text{obs}} } + \frac{f^{ \text{obs}^{2} } }{\bar{f}^{4} }\sigma^{2}_{\bar{f}}
$$

$$ 
 \sigma^{2}_{\text{tot} } =  \frac{f^{ \text{obs}^{2} } }{\bar{f}^{2} } \left( \frac{ \sigma^{2}_{f^{\text{obs}} }  }{{f}^{\text{obs}^{2}} }  +  \frac{ \sigma^{2}_{\bar{f}}  }{\bar{f}^{2} } \right)
$$

Finally, $ \sigma^{2}_{\bar{f}}$ is given in the same way:

$$ 
\sigma^{2}_{\bar{f}} =  \sum_{m} \, \left[ \frac{\partial \, H(X_{m})}{\partial X_{m}}\right]\sigma^{2}_{X_{n}}
$$

And thus,

$$ 
\sigma^{2}_{\bar{f}} = \left[ \frac{\partial }{\partial K  } \bar{f} \right]^{2} \sigma^{2}_{K} +  \left[ \frac{\partial }{\partial A  } \bar{f} \right]^{2} \sigma^{2}_{A} +  \left[ \frac{\partial }{\partial \Omega_{b}   } \bar{f} \right]^{2} \sigma^{2}_{\Omega_{b}} +  \left[ \frac{\partial }{\partial \Omega_{m}  } \bar{f} \right]^{2} \sigma^{2}_{\Omega_{m}} +  \left[ \frac{\partial }{\partial D^{*}_{A}  } \bar{f} \right]^{2} \sigma^{2}_{D^{*}_{A} } +  \left[ \frac{\partial }{\partial D_{A}   } \bar{f} \right]^{2} \sigma^{2}_{D_{A} } 
$$

Therefore, the error with respect to $\bar{f}$,

$$
\frac{\sigma^{2}_{ \bar{f} } }{\bar{f}^{2} } = \left[ \frac{ \sigma^{2}_{ K } }{ K^{2} } \, + \, \frac{ \sigma^{2}_{A} }{A^{2}} \, + \,  \frac{\sigma_{\gamma}^{2}}{\gamma^{2} }\, + \frac{ \sigma^{2}_{ \Omega_{b} } }{\Omega_{b}^{2}} \, + \, \sigma^{2}_{\Omega_{m}}\cdot \Omega_{m}^{2} \, + \, \left(\frac{2}{3}\right)^{2} \frac{\sigma^{2}_{D_{A} } }{D_{A}^{2} } \,    \right ]
$$

Taking care to remember that,

$$
 \sigma^{2}_{A}  =  \left( \frac{\partial A}{ \partial \theta } \right)^{2} \sigma^{2}_{\theta} 
$$

that is,

$$
\sigma^{2}_{A} = \left( \mbox{log}\, A\right)^{2} \sigma^{2}_{\theta}
$$

It can be concluded that the final expression for the total error in the first model is:

$$
\sigma^{2}_{\text{tot} } = \frac{f^{ \text{obs}^{2} } }{\bar{f}^{2} } \left[ \frac{ \sigma^{2}_{f^{\text{obs}} }  }{{f}^{\text{obs}^{2}} } \, + \, \frac{ \sigma^{2}_{ K } }{ K^{2} } \, + \, \frac{\sigma_{\gamma}^{2}}{\gamma^{2} }\, + \, \frac{\left( \mbox{log}\, A\right)^{2} \sigma^{2}_{\theta}}{A^{2}} \, + \,  \frac{ \sigma^{2}_{ \Omega_{b} } }{\Omega_{b}^{2}} \, + \, \sigma^{2}_{\Omega_{m}}\cdot \Omega_{m}^{2} \, + \, \left(\frac{2}{3}\right)^{2} \frac{\sigma^{2}_{D_{A} } }{D_{A}^{2} } \,    \right ]
$$

### Priors for the Model 1:

The priors on $\eta$ for the first model will be chosen as flat (uniform) priors. Despite being non-informative a priori, they have the advantage of not making any assumptions about the parameter except for the range of values.

$$
\ln \, \left[\,\mathcal{P}(\eta_{0})\,\right] = \left \{\begin{array}{ll}
        0 \,, & \mbox{ if }\, -5 < \eta_{0}\,  < 5. \\
        -\infty \,, & \mbox{ outro }
    \end{array}
    \right .
$$

where, 

$$
-2\ln\mathcal{P} =   \sum_{i=1}^{40}\frac{ \left[ (1+\eta _{0}z)^{2}-
\frac{f_{\text{gas},\ i}^{\text{X-ray} } }{ \bar{f}_{gas} }\,\,\,\,\, \right] ^{2}}{\sigma _{\text{
tot},\ i}^{2}}\,\, +\,\, \sum_{i=1}^{40}\ln 2\pi{\sigma_{\text{tot}, i}^2} 
$$

So we define the log_prior_mod1 function as follows:

In [2]:
def log_prior_mod1(gama_0, eta_0):
    '''
    log likelihood para uma prior não informativa, flat nos parâmetros.
    '''
    
    if  -3 < gama_0 < 3 and -3 < eta_0 < 3:
        return 0.0

    return -np.inf

and the $\chi^{2}$ 

In [6]:
def chi2_fgas(gama_0, eta_0, dA_fid, dA_obs, A, param_cosmo=param_cosmo_fid):
    '''
    chi2_Fgas é a função que calcula o chi-quadrado da fração de massa do gás em
    função dos parâmetros de interesse, gamma_0 e eta_0.
    '''
    
    # K(z) que é a "calibration Constant" no modelo da fração de massa do gás, e o erro =  ± 0.12
    Kz = 0.96
    sigma_Kz = 0.12

    # \gamma é o fator de depleção dos bárions, 
    gamma = 0.848
    sigma_gamma = 0.085

    sigma_omega_b = 0.0039
    sigma_omega_m = 0.0073

    # \theta , e o erro =  ± 0.035
    theta = 0.442
    sigma_theta = 0.035
        
    omega_c, omega_b, omega_k, h, w_x = param_cosmo
    
    frac_f_gas = f_gas/f_barra
    
    
    sigma_tot_2 = (frac_f_gas**2)*(  (f_gas_erro/f_gas)**2.0 +  (sigma_K_z/K_z)**2.0   + (sigma_gamma/gamma)**2.0
                                                                +(sigma_omega_b/omega_b)**2.0 +   (sigma_omega_m*omega_m)**2.0
                                                                + (sigma_theta*np.log(A)/A)**2.0 + ( (2/3)*(Da_Obs_erro/Da_Obs) )**2    )

    
    return   (  ( gama_0*(1+eta_0*z)**2 - frac_f_gas  )**2.0 ) / sigma_tot_2 + np.log(2*np.pi*sigma_tot_2)       